This notebook has some misc data cleaning and light data exploration. It is all over the place and the cells were run haphazardly, so do not use.

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('fight_data_clean.csv')

In [4]:
unique_values = df['method'].value_counts()
print(unique_values.sort_values(ascending=False))

method
Decision - Unanimous       2493
KO/TKO                     2203
Submission                 1344
Decision - Split            678
Decision - Majority          87
TKO - Doctor's Stoppage      80
Overturned                   50
Could Not Continue           22
DQ                           20
Other                         1
Name: count, dtype: int64


In [ ]:
import requests
from bs4 import BeautifulSoup

def extract_html(url):
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        html_content  = BeautifulSoup(response.content, 'html.parser')
        return html_content
    except Exception as e:
        print("Failed to fetch the URL", url, ":", e)

base_url = "http://www.ufcstats.com/statistics/events/completed?page=all"

try:
    html_content = extract_html(base_url)
    events = []

    for i in html_content.select('i.b-statistics__table-content'):
        a_tag = i.select_one('a.b-link.b-link_style_black')
        span_tag = i.select_one('span.b-statistics__date')
        
        if a_tag and span_tag:
            event_link = a_tag['href']
            event_name = a_tag.text.strip()
            event_date = span_tag.text.strip()

            events.append({
                'link': event_link,
                'name': event_name,
                'date': event_date,
            })

except Exception as e:
    print("Failed to scrape the base url:", e)


In [5]:
df_dates = pd.read_csv('fight_card_dates.csv')
df_dates = df_dates.rename({'name':'event_name'}, axis=1)

In [6]:
merged_df = pd.merge(df, df_dates, on='event_name', how='left')

In [7]:
merged_df.columns

Index(['fight_link', 'event_name', 'first_fighter_name', 'second_fighter_name',
       'first_fighter_KD', 'second_fighter_KD', 'first_fighter_STR',
       'second_fighter_STR', 'first_fighter_TD', 'second_fighter_TD',
       ...
       'first_fighter_round_5_CLINCH_attempted',
       'second_fighter_round_5_CLINCH_landed',
       'second_fighter_round_5_CLINCH_attempted',
       'first_fighter_round_5_GROUND_landed',
       'first_fighter_round_5_GROUND_attempted',
       'second_fighter_round_5_GROUND_landed',
       'second_fighter_round_5_GROUND_attempted', 'Unnamed: 0', 'link',
       'date'],
      dtype='object', length=239)

In [ ]:
merged_df.loc[:, ['first_fighter_name', 'second_fighter_name', 'event_name', 'date']]

In [27]:
import plotly.express as px

# Define your data
data = {
    'total': ['Total']*7,
    'outcome': ['Decision', 'Decision', 'Decision', 'Finish', 'Finish', 'Finish', 'Finish'],
    'type': ['Split Decision', 'Majority Decision', 'Unanimous Decision', 'KO/TKO', 'Submission', 'Doctor\'s Stoppage', 'Could Not Continue'],
    'value': [0.098161286, 0.012595917, 0.360938179, 0.318951788, 0.194585203, 0.011582453, 0.003185174]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Create the sunburst chart
fig = px.sunburst(df, path=['total', 'outcome', 'type'], values='value',
                  template = 'seaborn',
                  width=600, height=600)

fig.update_layout(
    margin = dict(t=10, l=10, r=10, b=10)
)

# Display the figure
fig.show()


In [21]:
# Define your data
data = {
    'total': ['Total']*7,
    'outcome': ['Decision', 'Decision', 'Decision', 'Finish', 'Finish', 'Finish', 'Finish'],
    'type': ['Decision - Split', 'Decision - Majority', 'Decision - Unanimous', 'KO/TKO', 'Submission', 'TKO - Doctor\'s Stoppage', 'Could Not Continue'],
    'value': [0.098161286, 0.012595917, 0.360938179, 0.318951788, 0.194585203, 0.011582453, 0.003185174]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Create the treemap
fig = px.treemap(df, path=['total', 'outcome', 'type'], values='value',
                 title='Outcome Distribution',
                 width=600, height=600)

fig.update_traces(textinfo="label+percent root",
                  textfont_size=12,
                  marker=dict(line=dict(color='#000000', width=2)))

fig.update_layout(
    margin = dict(t=50, l=0, r=0, b=0)
)

# Display the figure
fig.show()


In [3]:
# Add a column to represent the winner, with the winner as 1 and the loser as 0
df['winner'] = 1

# Copy the dataframe and flip the a/b columns
df_flipped = df.copy()
df_flipped.columns = df.columns.str.replace('first_', 'temp_').str.replace('second_', 'first_').str.replace('temp_', 'second_')
df_flipped['winner'] = 0

# Concatenate the original and flipped dataframes
df = pd.concat([df, df_flipped])

# Randomly shuffle the rows
df = df.sample(frac=1, random_state=42).reset_index(drop=True)